In [43]:
import pandas as pd
import folium
from folium.plugins import HeatMap
from folium.plugins import HeatMapWithTime
import numpy as np
from sklearn.cluster import MeanShift
from sklearn.datasets.samples_generator import make_blobs
from mpl_toolkits.mplot3d import Axes3D
%matplotlib notebook
import matplotlib.pyplot as plt
from matplotlib import style
style.use("ggplot")


df = pd.read_csv('asd.csv')

/Users/zarif/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [44]:
#Preprocessing to focus on Austin data points and to remove and bicycle data points

df.drop(df[df['Start Longitude'] < -98.01118].index, inplace=True)
df.drop(df[df['Start Latitude'] > 30.51436].index, inplace=True)
df.drop(df[df['Start Longitude'] > -97.45534].index, inplace=True)
df.drop(df[df['Start Latitude'] < 30.10504].index, inplace=True)
df.drop(df[df['End Longitude'] < -98.01118].index, inplace=True)
df.drop(df[df['End Latitude'] > 30.51436].index, inplace=True)
df.drop(df[df['End Longitude'] > -97.45534].index, inplace=True)
df.drop(df[df['End Latitude'] < 30.10504].index, inplace=True)
df.drop(df[df['Vehicle Type'] == 'bicycle'].index, inplace=True)

#preprocessing to convert measurements
df['Trip Distance (Miles)'] = (df['Trip Distance']/1000)*0.621371
df.rename(columns={'Trip Duration': 'Trip Duration (Seconds)', 'Trip Distance': 'Trip Distance (Meters)'}, inplace = True)
df.drop(df[df['Trip Distance (Meters)'] < 0].index, inplace=True)
df.drop(df[df['Trip Distance (Miles)'] > 75].index, inplace=True)

#drop any null values
df = df.dropna(how='any',axis=0) 

df.head()

,ID,Device ID,Vehicle Type,Trip Duration (Seconds),Trip Distance (Meters),Start Time,End Time,Modified Date,Month,Hour,...,Council District (Start),Council District (End),Origin Cell ID,Destination Cell ID,Year,Start Latitude,Start Longitude,End Latitude,End Longitude,Trip Distance (Miles)
0,000000cb-78a9-4b95-8ead-37c797abcf37,ad14e63b-940d-4ca3-84c6-0131147fba45,scooter,358.0,915.0,06/29/2018 05:30:00 PM,06/29/2018 05:30:00 PM,12/12/2018 09:36:14 PM,6.0,17.0,...,5.0,9.0,013445,013916,2018.0,30.263,-97.762,30.259,-97.754,0.568554
1,00001b5f-46aa-4b26-a12f-d7b4d3c69834,c5a230c9-7e80-4464-862f-f2b23f61d6d3,scooter,226.0,839.0,11/30/2018 11:00:00 AM,11/30/2018 11:00:00 AM,12/13/2018 03:43:44 PM,11.0,11.0,...,9.0,9.0,014556,015186,2018.0,30.283,-97.745,30.284,-97.737,0.521330
2,00002334-2f2b-41a0-9a04-8ff66be39f40,5a9f7815-bf0c-4fef-8bac-60efd4f23b10,scooter,324.0,1206.0,10/02/2018 03:15:00 PM,10/02/2018 03:30:00 PM,12/13/2018 03:49:54 PM,10.0,15.0,...,9.0,9.0,015344,015032,2018.0,30.285,-97.734,30.291,-97.738,0.749373
3,0000261e-c0f0-4fb1-9947-1606dfe4ddcf,41fa0cf0-f433-4b3a-89f5-2c6cec6c367d,scooter,1096.0,0.0,09/02/2018 04:45:00 PM,09/02/2018 05:00:00 PM,12/13/2018 03:53:09 PM,9.0,16.0,...,9.0,9.0,014707,014081,2018.0,30.268,-97.743,30.276,-97.752,0.000000
4,00002c70-678b-4c09-a32c-da0266e329c6,ddd4bbc3-114e-4cbc-b078-860e3c28b11b,scooter,408.0,1144.0,09/28/2018 11:45:00 AM,09/28/2018 11:45:00 AM,12/12/2018 10:24:56 PM,9.0,11.0,...,3.0,3.0,016114,015642,2018.0,30.242,-97.721,30.244,-97.729,0.710848


In [45]:
#function to generate map of Austin

def generateBaseMap(default_location=[30.2672, -97.7431], default_zoom_start=12):
    base_map = folium.Map(location=default_location, control_scale=True, zoom_start=default_zoom_start)
    return base_map

In [46]:
#Heat Map to identify the concentration of starting dockless rides

df_copy = df.copy()
df_copy['count'] = 1
base_map = generateBaseMap()


HeatMap(data=df_copy[['Start Latitude', 'Start Longitude', 'count']].groupby(['Start Latitude', 'Start Longitude']).sum().reset_index().values.tolist(), radius=8, max_zoom=13).add_to(base_map)


base_map



In [47]:
#Heat Map to identify the concentration of ending dockless rides


df_copy = df.copy()
df_copy['count'] = 1
base_map = generateBaseMap()


HeatMap(data=df_copy[['End Latitude', 'End Longitude', 'count']].groupby(['End Latitude', 'End Longitude']).sum().reset_index().values.tolist(), radius=8, max_zoom=13).add_to(base_map)


base_map

In [48]:
#Hierarchial Clustering for scooter starting points

copy = df.drop(columns=['ID', 'Device ID', 'Vehicle Type', 'Start Time', 'End Time', 'Modified Date', 'Origin Cell ID', 'Destination Cell ID', 'Trip Duration (Seconds)', 'Trip Distance (Meters)', 'Trip Distance (Miles)', 'Month', 'Hour', 'Day of Week', 'Council District (Start)', 'Council District (End)', 'Year', 'End Latitude', 'End Longitude'])


start_df = copy.sample(n = 10000)

start_df.head()



,Start Latitude,Start Longitude
559255,30.267,-97.740
2024047,30.266,-97.745
882633,30.260,-97.748
1734640,30.262,-97.740
372507,30.265,-97.739


In [49]:
#scatter plot of sampled data for starting points
plt.scatter(start_df.iloc[:,0],start_df.iloc[:,1])
plt.show()

<IPython.core.display.Javascript object>

In [50]:
ms = MeanShift()
ms.fit(start_df)
labels = ms.labels_
cluster_centers = ms.cluster_centers_

In [51]:
print(cluster_centers)
n_clusters_ = len(np.unique(labels))
print("Number of estimated clusters:", n_clusters_)

[[ 30.26547831 -97.74357816]
 [ 30.28300553 -97.74078822]
 [ 30.29799398 -97.70471687]
 [ 30.32746774 -97.72320968]
 [ 30.39818644 -97.71971186]
 [ 30.36866667 -97.71606667]
 [ 30.3645     -97.69691667]
 [ 30.404875   -97.69975   ]
 [ 30.3535     -97.67583333]
 [ 30.36983333 -97.75416667]
 [ 30.4012     -97.7426    ]
 [ 30.2054     -97.7958    ]
 [ 30.198      -97.77775   ]
 [ 30.34075    -97.69525   ]
 [ 30.16066667 -97.83066667]
 [ 30.371      -97.785     ]
 [ 30.298      -97.624     ]
 [ 30.28       -97.669     ]
 [ 30.21       -97.652     ]
 [ 30.431      -97.634     ]
 [ 30.165      -97.789     ]
 [ 30.233      -97.836     ]
 [ 30.467      -97.789     ]]
Number of estimated clusters: 23


In [52]:
clusters_centers_df = pd.DataFrame(data=cluster_centers)

In [53]:
clusters_centers_df.head()

,0,1
0,30.265478,-97.743578
1,30.283006,-97.740788
2,30.297994,-97.704717
3,30.327468,-97.723210
4,30.398186,-97.719712


In [54]:
Austin_Map = folium.Map(location = [30.267153, -97.7430608], zoom_start = 12)

for i in range (0, len(cluster_centers)):
    description = 'CLUSTER #' + str(i + 1)
    folium.Marker([cluster_centers[i][0], cluster_centers[i][1]], popup=folium.Popup(description, max_width = 110), icon=folium.Icon(color='lightblue')).add_to(Austin_Map)

Austin_Map.save("./Cluster_Centers_Start_Location.html")



In [55]:
colors = 10*['r.','g.','b.','c.','k.','y.','m.']


In [56]:
for i in range(len(start_df)):
    plt.plot(start_df.iloc[i][0], start_df.iloc[i][1], colors[labels[i]], markersize = 10)

plt.scatter(cluster_centers[:,0],cluster_centers[:,1],
            marker="x",color='k', s=150, linewidths = 5, zorder=10)

plt.show()

In [57]:
#Hierarchial Clustering for scooter ending points


copy = df.drop(columns=['ID', 'Device ID', 'Vehicle Type', 'Start Time', 'End Time', 'Modified Date', 'Origin Cell ID', 'Destination Cell ID', 'Trip Duration (Seconds)', 'Trip Distance (Meters)', 'Trip Distance (Miles)', 'Month', 'Hour', 'Day of Week', 'Council District (Start)', 'Council District (End)', 'Year', 'Start Latitude', 'Start Longitude'])


end_df = copy.sample(n = 10000)

end_df.head()

,End Latitude,End Longitude
887805,30.286,-97.744
2447160,30.266,-97.743
2520511,30.267,-97.751
1172587,30.265,-97.708
459377,30.291,-97.742


In [58]:
#scatter plot of sampled data for ending points
plt.scatter(end_df.iloc[:,0],end_df.iloc[:,1])
plt.show()

In [59]:
ms = MeanShift()
ms.fit(end_df)
labels = ms.labels_
cluster_centers = ms.cluster_centers_

In [60]:
print(cluster_centers)
n_clusters_ = len(np.unique(labels))
print("Number of estimated clusters:", n_clusters_)

[[ 30.26583006 -97.74351685]
 [ 30.2842036  -97.74024907]
 [ 30.29620213 -97.70449468]
 [ 30.39896226 -97.72003774]
 [ 30.34024138 -97.72941379]
 [ 30.22576923 -97.698     ]
 [ 30.37663636 -97.71672727]
 [ 30.20855556 -97.77666667]
 [ 30.20925    -97.81125   ]
 [ 30.2815     -97.668     ]
 [ 30.36175    -97.76375   ]
 [ 30.40233333 -97.66533333]
 [ 30.1565     -97.6465    ]
 [ 30.381      -97.663     ]
 [ 30.381      -97.687     ]
 [ 30.428      -97.61      ]
 [ 30.204      -97.834     ]
 [ 30.231      -97.85      ]
 [ 30.426      -97.719     ]
 [ 30.161      -97.791     ]
 [ 30.33       -97.817     ]
 [ 30.186      -97.758     ]
 [ 30.427      -97.759     ]]
Number of estimated clusters: 23


In [61]:
clusters_centers_df = pd.DataFrame(data=cluster_centers)

In [62]:
clusters_centers_df.head()

,0,1
0,30.265830,-97.743517
1,30.284204,-97.740249
2,30.296202,-97.704495
3,30.398962,-97.720038
4,30.340241,-97.729414


In [63]:
Austin_Map = folium.Map(location = [30.267153, -97.7430608], zoom_start = 12)

for i in range (0, len(cluster_centers)):
    description = 'CLUSTER #' + str(i + 1)
    folium.Marker([cluster_centers[i][0], cluster_centers[i][1]], popup=folium.Popup(description, max_width = 110), icon=folium.Icon(color='lightblue')).add_to(Austin_Map)

Austin_Map.save("./Cluster_Centers_End_Location.html")

In [64]:
for i in range(len(end_df)):
    plt.plot(end_df.iloc[i][0], end_df.iloc[i][1], colors[labels[i]], markersize = 10)

plt.scatter(cluster_centers[:,0],cluster_centers[:,1],
            marker="x",color='k', s=150, linewidths = 5, zorder=10)

plt.show()

In [83]:
# 3-D Hierarchial Clustering for hour, day of week, and month to find when scooters are most used

copy = df.drop(columns=['ID', 'Device ID', 'Vehicle Type', 'Start Time', 'End Time', 'Modified Date', 'Origin Cell ID', 'Destination Cell ID', 'Trip Duration (Seconds)', 'Trip Distance (Meters)', 'Trip Distance (Miles)', 'Council District (Start)', 'Council District (End)', 'Year', 'Start Latitude', 'Start Longitude', 'End Latitude', 'End Longitude'])


time_df = copy.sample(n = 10000)

time_df.head()

,Month,Hour,Day of Week
1509118,9.0,10.0,6.0
2175402,1.0,18.0,5.0
2012004,9.0,8.0,3.0
948466,11.0,9.0,2.0
1922649,10.0,17.0,6.0


In [84]:
ms = MeanShift()
ms.fit(time_df)
labels = ms.labels_
cluster_centers = ms.cluster_centers_

print(cluster_centers)

[[ 9.71856474 15.04617785  3.20639626]]


In [85]:
n_clusters_ = len(np.unique(labels))
colors = 10*['r','g','b','c','k','y','m']

print("Number of estimated clusters:", n_clusters_)

Number of estimated clusters: 1


In [86]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.set_xlabel('Month')
ax.set_ylabel('Hour')
ax.set_zlabel('Day of Week')

<IPython.core.display.Javascript object>

Text(0.5,0,'Day of Week')

In [91]:
for i in range(len(time_df)):

    ax.scatter(time_df.iloc[i][0], time_df.iloc[i][1], time_df.iloc[i][2], c=colors[labels[i]], marker='o')
    
    
ax.scatter(cluster_centers[:,0],cluster_centers[:,1],cluster_centers[:,2],
            marker="x",color='k', s=150, linewidths = 5, zorder=10)

fig.show()

#ax.save("./3dtime.html")
fig.savefig("three-time.html")

AttributeError: 'FigureCanvasNbAgg' object has no attribute 'manager'
Figure.show works only for figures managed by pyplot, normally created by pyplot.figure().

Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x109e6bea0> (for post_execute):


AttributeError: 'FigureCanvasNbAgg' object has no attribute 'manager'

In [70]:
# 3-D Hierarchial Clustering for Duration and Distance

copy = df.drop(columns=['ID', 'Device ID', 'Vehicle Type', 'Start Time', 'End Time', 'Modified Date', 'Origin Cell ID', 'Destination Cell ID', 'Trip Distance (Meters)', 'Month', 'Hour', 'Day of Week', 'Council District (Start)', 'Council District (End)', 'Year', 'Start Latitude', 'Start Longitude','End Latitude', 'End Longitude'])


duration_df = copy.sample(n = 10000)

duration_df.head()

,Trip Duration (Seconds),Trip Distance (Miles)
2107672,464.0,0.536865
338096,1518.0,3.638127
1793348,1101.0,2.126953
1074918,176.0,0.329327
2457321,245.0,0.419425


In [71]:
ms = MeanShift()
ms.fit(duration_df)
labels = ms.labels_
cluster_centers = ms.cluster_centers_

In [72]:
n_clusters_ = len(np.unique(labels))
colors = 10*['r','g','b','c','k','y','m']

print("Number of estimated clusters:", n_clusters_)

Number of estimated clusters: 14


In [73]:
print(cluster_centers)
n_clusters_ = len(np.unique(labels))
print("Number of estimated clusters:", n_clusters_)

[[3.07085695e+02 4.52368824e-01]
 [3.23065455e+03 3.61371862e+00]
 [4.31419565e+03 4.12187804e+00]
 [5.95414286e+03 4.67679322e+00]
 [5.51407692e+03 3.27591571e+00]
 [6.97166667e+03 4.05672414e+00]
 [7.50200000e+03 3.48029897e+00]
 [8.53700000e+03 6.14991591e+00]
 [1.11433333e+04 9.07615907e+00]
 [3.74930000e+04 0.00000000e+00]
 [1.33010000e+04 7.32161449e+00]
 [1.18340000e+04 6.62319349e+00]
 [1.05590000e+04 7.03329835e+00]
 [9.94700000e+03 5.38604383e+00]]
Number of estimated clusters: 14


In [74]:
for i in range(len(duration_df)):
    plt.plot(duration_df.iloc[i][0], duration_df.iloc[i][1], colors[labels[i]], markersize = 10)

plt.scatter(cluster_centers[:,0],cluster_centers[:,1],
            marker="x",color='k', s=150, linewidths = 5, zorder=10)

plt.xlabel('Trip Duration (Seconds)')
plt.ylabel('Trip Distance (Miles)')
plt.show()

TypeError: object of type 'numpy.float64' has no len()

In [75]:
copy = df.drop(columns=['ID', 'Device ID', 'Vehicle Type', 'Origin Cell ID', 'Destination Cell ID',  'Council District (Start)', 'Council District (End)', 'Year', 'End Latitude', 'End Longitude'])
copy.head()

,Trip Duration (Seconds),Trip Distance (Meters),Start Time,End Time,Modified Date,Month,Hour,Day of Week,Start Latitude,Start Longitude,Trip Distance (Miles)
0,358.0,915.0,06/29/2018 05:30:00 PM,06/29/2018 05:30:00 PM,12/12/2018 09:36:14 PM,6.0,17.0,5.0,30.263,-97.762,0.568554
1,226.0,839.0,11/30/2018 11:00:00 AM,11/30/2018 11:00:00 AM,12/13/2018 03:43:44 PM,11.0,11.0,5.0,30.283,-97.745,0.521330
2,324.0,1206.0,10/02/2018 03:15:00 PM,10/02/2018 03:30:00 PM,12/13/2018 03:49:54 PM,10.0,15.0,2.0,30.285,-97.734,0.749373
3,1096.0,0.0,09/02/2018 04:45:00 PM,09/02/2018 05:00:00 PM,12/13/2018 03:53:09 PM,9.0,16.0,0.0,30.268,-97.743,0.000000
4,408.0,1144.0,09/28/2018 11:45:00 AM,09/28/2018 11:45:00 AM,12/12/2018 10:24:56 PM,9.0,11.0,5.0,30.242,-97.721,0.710848


In [76]:
df_hour_list = []
for hour in df['Month'].sort_values().unique():
    df_hour_list.append(df.loc[df['Month'] == hour, ['Start Latitude', 'Start Longitude', 'count']].groupby(['Start Latitude', 'Start Longitude']).sum().reset_index().values.tolist())


/Users/zarif/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


In [77]:
from folium.plugins import HeatMapWithTime
base_map = generateBaseMap(default_zoom_start=11)
HeatMapWithTime(df_hour_list, radius=5, gradient={0.2: 'blue', 0.4: 'lime', 0.6: 'orange', 1: 'red'}, min_opacity=0.5, max_opacity=0.8, use_local_extrema=True).add_to(base_map)
base_map

base_map.save('time_map.html')

